<a href="https://colab.research.google.com/github/v4760/face_detection_neural_nets_YOLO/blob/main/Second_Submission_YOLOV8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r "/content/drive/MyDrive/requirements.txt" -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 866.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:


# Libraries
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.metrics import accuracy_score
import pickle

from ultralytics import YOLO
from ultralytics.engine.results import Results
from deepface import DeepFace
import gradio as gr
import shutil
import pandas
import cv2
import os

from PIL import Image, ImageEnhance, ImageOps
import random


from sklearn.model_selection import train_test_split


from tqdm import tqdm

import re


import json
from datetime import datetime

print("✓ Libraries installed successfully!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
25-10-06 12:25:51 - Directory /root/.deepface has been created
25-10-06 12:25:51 - Directory /root/.deepface/weights has been created
✓ Libraries installed successfully!


Mounted at /content/drive


In [ ]:

def augment_image(img):
    """Apply random augmentation to an image"""
    augmentations = [
        lambda x: x.rotate(random.uniform(-15, 15), fillcolor=(0, 0, 0)),  # Random rotation
        lambda x: ImageEnhance.Brightness(x).enhance(random.uniform(0.8, 1.2)),  # Brightness
        lambda x: ImageEnhance.Contrast(x).enhance(random.uniform(0.8, 1.2)),  # Contrast
        lambda x: x.transpose(Image.FLIP_LEFT_RIGHT),  # Horizontal flip
        lambda x: ImageOps.autocontrast(x),  # Auto contrast
        lambda x: x.rotate(random.uniform(-10, 10), fillcolor=(0, 0, 0)),  # Slight rotation
    ]

    # Apply 1-2 random augmentations
    num_augs = random.randint(1, 2)
    selected_augs = random.sample(augmentations, num_augs)

    augmented = img.copy()
    for aug in selected_augs:
        augmented = aug(augmented)

    return augmented

def process_identity_folders(base_folder, target_count=100):
    """Process each identity folder and augment to reach target count"""

    # Get all subdirectories (identity folders)
    identity_folders = [f for f in os.listdir(base_folder)
                       if os.path.isdir(os.path.join(base_folder, f))]

    print(f"Found {len(identity_folders)} identity folders")

    for identity_folder in identity_folders:
        folder_path = os.path.join(base_folder, identity_folder)

        # Get all image files in the folder
        image_files = [f for f in os.listdir(folder_path)
                      if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

        current_count = len(image_files)

        if current_count >= target_count:
            print(f"✓ {identity_folder}: {current_count} images (no augmentation needed)")
            continue

        needed = target_count - current_count
        print(f"→ {identity_folder}: {current_count} images, need {needed} more")

        # Load existing images
        images = []
        for img_file in image_files:
            try:
                img_path = os.path.join(folder_path, img_file)
                img = Image.open(img_path).convert('RGB')
                images.append(img)
            except Exception as e:
                print(f"  Warning: Could not load {img_file}: {e}")

        if len(images) == 0:
            print(f"  Error: No valid images in {identity_folder}")
            continue

        # Generate augmented images
        augmented_count = 0
        while augmented_count < needed:
            # Randomly select a source image
            source_img = random.choice(images)

            # Augment it
            augmented_img = augment_image(source_img)

            # Save with a unique name
            aug_filename = f"aug_{augmented_count:04d}.jpg"
            aug_path = os.path.join(folder_path, aug_filename)

            augmented_img.save(aug_path, quality=95)
            augmented_count += 1

        print(f"  ✓ Generated {augmented_count} augmented images")

        # Verify final count
        final_count = len([f for f in os.listdir(folder_path)
                          if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))])
        print(f"  Final count: {final_count} images")

# Usage
base_folder = '/content/drive/MyDrive/Celebrity Image Subsets'  # Change this to your folder path
process_identity_folders(base_folder, target_count=100)
print("\nAugmentation complete!")

Found 45 identity folders
✓ images__9151_: 100 images (no augmentation needed)
✓ images__800_: 100 images (no augmentation needed)
✓ images__9319_: 100 images (no augmentation needed)
✓ images__9152_: 100 images (no augmentation needed)
✓ images__8945_: 100 images (no augmentation needed)
✓ images__8871_: 100 images (no augmentation needed)
✓ images__228_: 100 images (no augmentation needed)
✓ images__7282_: 100 images (no augmentation needed)
✓ images__3401_: 100 images (no augmentation needed)
✓ images__3782_: 100 images (no augmentation needed)
✓ images__3321_: 100 images (no augmentation needed)
✓ images__2463_: 100 images (no augmentation needed)
✓ images__3699_: 100 images (no augmentation needed)
✓ images__10046_: 100 images (no augmentation needed)
✓ images__6098_: 100 images (no augmentation needed)
✓ images__3431_: 100 images (no augmentation needed)
✓ images__487_: 100 images (no augmentation needed)
✓ images__5239_: 100 images (no augmentation needed)
✓ images__4304_: 100 i

In [ ]:
def rebuild_dataset(base_folder,
                              output_folder='/content/dataset_rebuilt',
                              test_size=0.2,
                              val_size=0.1,
                              random_state=42):
    """Completely rebuild dataset with verified consistency"""

    random.seed(random_state)

    # Remove old output
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)

    print("="*70)
    print("REBUILDING DATASET WITH VERIFIED CONSISTENCY")
    print("="*70)

    # Create folders
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_folder, split), exist_ok=True)

    # Get all identity folders - SORT CONSISTENTLY
    identity_folders = sorted([f for f in os.listdir(base_folder)
                              if os.path.isdir(os.path.join(base_folder, f))])

    print(f"\nProcessing {len(identity_folders)} identities")
    print(f"First 5: {identity_folders[:5]}")
    print(f"Last 5: {identity_folders[-5:]}\n")

    # Create mapping
    import json
    mapping = {}

    # Process EACH identity with SAME class index for ALL splits
    for class_idx, identity_folder in enumerate(identity_folders):
        identity_path = os.path.join(base_folder, identity_folder)
        mapping[class_idx] = identity_folder

        # Get all images
        all_images = [f for f in os.listdir(identity_path)
                     if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        if len(all_images) == 0:
            print(f"⚠️  Warning: No images in {identity_folder}")
            continue

        # Split images
        train_val, test_imgs = train_test_split(
            all_images, test_size=test_size, random_state=random_state
        )

        val_size_adj = val_size / (1 - test_size)
        train_imgs, val_imgs = train_test_split(
            train_val, test_size=val_size_adj, random_state=random_state
        )

        # Create class folders - SAME INDEX for all splits!
        for split, images in [('train', train_imgs), ('val', val_imgs), ('test', test_imgs)]:
            # Use str(class_idx) for ALL splits
            class_folder = os.path.join(output_folder, split, str(class_idx))
            os.makedirs(class_folder, exist_ok=True)

            for img in images:
                src = os.path.join(identity_path, img)
                dst = os.path.join(class_folder, img)
                shutil.copy2(src, dst)

        if (class_idx + 1) % 10 == 0:
            print(f"Processed {class_idx + 1}/{len(identity_folders)} identities")

    # Save mapping
    mapping_file = os.path.join(output_folder, 'class_mapping.json')
    with open(mapping_file, 'w') as f:
        json.dump(mapping, f, indent=2)

    # VERIFY consistency
    print("\n" + "="*70)
    print("VERIFYING CONSISTENCY")
    print("="*70)

    for split in ['train', 'val', 'test']:
        split_path = os.path.join(output_folder, split)
        classes = sorted([f for f in os.listdir(split_path)
                         if os.path.isdir(os.path.join(split_path, f))],
                        key=lambda x: int(x))
        print(f"{split}: {len(classes)} classes ({classes[0]} to {classes[-1]})")

    # Count images
    total_train = sum(len([f for f in os.listdir(os.path.join(output_folder, 'train', c))
                          if f.endswith(('.jpg', '.jpeg', '.png'))])
                     for c in os.listdir(os.path.join(output_folder, 'train')))

    total_test = sum(len([f for f in os.listdir(os.path.join(output_folder, 'test', c))
                         if f.endswith(('.jpg', '.jpeg', '.png'))])
                    for c in os.listdir(os.path.join(output_folder, 'test')))

    print(f"\nTotal images:")
    print(f"  Train: {total_train}")
    print(f"  Test: {total_test}")

    print("\n" + "="*70)
    print(f"✅ DATASET REBUILT: {output_folder}")
    print("="*70)

    return output_folder

# Rebuild dataset
base_folder = '/content/drive/MyDrive/Celebrity Image Subsets'
new_dataset = rebuild_dataset(base_folder)

REBUILDING DATASET WITH VERIFIED CONSISTENCY

Processing 45 identities
First 5: ['images_7904_', 'images__10046_', 'images__10173_', 'images__1158_', 'images__1499_']
Last 5: ['images__9063_', 'images__9151_', 'images__9152_', 'images__9256_', 'images__9319_']

Processed 10/45 identities
⚠️  Warning: No images in images__2562_
Processed 20/45 identities
Processed 30/45 identities
Processed 40/45 identities

VERIFYING CONSISTENCY
train: 44 classes (0 to 44)
val: 44 classes (0 to 44)
test: 44 classes (0 to 44)

Total images:
  Train: 3080
  Test: 880

✅ DATASET REBUILT: /content/dataset_rebuilt


In [ ]:
class FaceRecognitionSystem:
    """Complete face recognition system using MTCNN + FaceNet"""

    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        print(f"Using device: {self.device}")

        # Face detector (MTCNN - better than YOLO for faces)
        self.detector = MTCNN(
            image_size=160,
            margin=0,
            min_face_size=20,
            thresholds=[0.6, 0.7, 0.7],
            factor=0.709,
            post_process=True,
            device=self.device
        )

        # Face recognizer (FaceNet)
        self.recognizer = InceptionResnetV1(pretrained='vggface2').eval().to(self.device)

        # Classifier and encoder (will be trained)
        self.classifier = None
        self.label_encoder = None
        self.normalizer = Normalizer(norm='l2')

    def extract_face_embedding(self, image_path):
        """Extract face embedding from image"""
        try:
            img = Image.open(image_path).convert('RGB')

            # Detect face
            face = self.detector(img)

            if face is None:
                return None

            # Get embedding
            face = face.unsqueeze(0).to(self.device)

            with torch.no_grad():
                embedding = self.recognizer(face).cpu().numpy()

            return embedding.flatten()

        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            return None

    def prepare_training_data(self, dataset_path):
        """Extract embeddings from training data"""

        print("Extracting face embeddings from training data...")

        train_path = os.path.join(dataset_path, 'train')

        embeddings = []
        labels = []

        # Get all class folders
        class_folders = sorted([f for f in os.listdir(train_path)
                               if os.path.isdir(os.path.join(train_path, f))],
                              key=lambda x: int(x))

        print(f"Processing {len(class_folders)} classes...\n")

        for class_idx in tqdm(class_folders, desc="Classes"):
            class_path = os.path.join(train_path, class_idx)

            # Get all images
            images = [f for f in os.listdir(class_path)
                     if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

            for img_file in images:
                img_path = os.path.join(class_path, img_file)

                # Extract embedding
                embedding = self.extract_face_embedding(img_path)

                if embedding is not None:
                    embeddings.append(embedding)
                    labels.append(int(class_idx))

        embeddings = np.array(embeddings)
        labels = np.array(labels)

        print(f"\n✓ Extracted {len(embeddings)} face embeddings")
        print(f"  Embedding shape: {embeddings.shape}")
        print(f"  Classes: {len(np.unique(labels))}")

        return embeddings, labels

    def train_classifier(self, embeddings, labels):
        """Train SVM classifier on face embeddings"""

        print("\nTraining classifier...")

        # Normalize embeddings
        embeddings_normalized = self.normalizer.transform(embeddings)

        # Encode labels
        self.label_encoder = LabelEncoder()
        labels_encoded = self.label_encoder.fit_transform(labels)

        # Train SVM
        self.classifier = SVC(
            kernel='linear',
            probability=True,
            C=1.0,
            random_state=42
        )

        self.classifier.fit(embeddings_normalized, labels_encoded)

        # Check training accuracy
        train_pred = self.classifier.predict(embeddings_normalized)
        train_acc = accuracy_score(labels_encoded, train_pred)

        print(f"✓ Classifier trained!")
        print(f"  Training accuracy: {train_acc * 100:.2f}%")

        return train_acc

    def predict(self, image_path):
        """Predict identity from image"""

        # Extract embedding
        embedding = self.extract_face_embedding(image_path)

        if embedding is None:
            return None, None

        # Normalize
        embedding_normalized = self.normalizer.transform([embedding])

        # Predict
        pred_encoded = self.classifier.predict(embedding_normalized)[0]
        pred_proba = self.classifier.predict_proba(embedding_normalized)[0]

        # Decode
        pred_class = self.label_encoder.inverse_transform([pred_encoded])[0]
        confidence = pred_proba[pred_encoded]

        return pred_class, confidence

    def evaluate(self, dataset_path, split='test'):
        """Evaluate on test set"""

        print(f"\nEvaluating on {split} set...")

        split_path = os.path.join(dataset_path, split)

        class_folders = sorted([f for f in os.listdir(split_path)
                               if os.path.isdir(os.path.join(split_path, f))],
                              key=lambda x: int(x))

        correct = 0
        total = 0

        for class_idx in tqdm(class_folders, desc="Testing"):
            class_path = os.path.join(split_path, class_idx)

            images = [f for f in os.listdir(class_path)
                     if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

            for img_file in images:
                img_path = os.path.join(class_path, img_file)

                pred_class, confidence = self.predict(img_path)

                if pred_class is not None:
                    total += 1
                    if pred_class == int(class_idx):
                        correct += 1

        accuracy = (correct / total * 100) if total > 0 else 0

        print(f"\n{'='*70}")
        print(f"{split.upper()} SET RESULTS")
        print(f"{'='*70}")
        print(f"Total images: {total}")
        print(f"Correct: {correct}")
        print(f"Accuracy: {correct}/{total} = {accuracy:.2f}%")
        print(f"{'='*70}")

        return accuracy

    def save_model(self, filepath='face_recognition_model.pkl'):
        """Save trained classifier"""
        model_data = {
            'classifier': self.classifier,
            'label_encoder': self.label_encoder,
            'normalizer': self.normalizer
        }

        with open(filepath, 'wb') as f:
            pickle.dump(model_data, f)

        print(f"✓ Model saved to {filepath}")

    def load_model(self, filepath='face_recognition_model.pkl'):
        """Load trained classifier"""
        with open(filepath, 'rb') as f:
            model_data = pickle.load(f)

        self.classifier = model_data['classifier']
        self.label_encoder = model_data['label_encoder']
        self.normalizer = model_data['normalizer']

        print(f"✓ Model loaded from {filepath}")

# Initialize system
print("="*70)
print("INITIALIZING FACE RECOGNITION SYSTEM")
print("="*70)
face_system = FaceRecognitionSystem()

INITIALIZING FACE RECOGNITION SYSTEM
Using device: cuda


  0%|          | 0.00/107M [00:00<?, ?B/s]

TRAINING AND TESTING THE MODEL WITH SINGLE IMAGES

In [ ]:
# Dataset path
dataset_path = '/content/dataset_rebuilt'  # Use your correct dataset

# Step 1: Extract embeddings from training data
embeddings, labels = face_system.prepare_training_data(dataset_path)

# Step 2: Train classifier
train_accuracy = face_system.train_classifier(embeddings, labels)

# Step 3: Evaluate on test set
test_accuracy = face_system.evaluate(dataset_path, split='test')

# Step 4: Save model
face_system.save_model('/content/face_recognition_facenet.pkl')

Extracting face embeddings from training data...
Processing 44 classes...



Classes: 100%|██████████| 44/44 [02:20<00:00,  3.20s/it]



✓ Extracted 3078 face embeddings
  Embedding shape: (3078, 512)
  Classes: 44

Training classifier...
✓ Classifier trained!
  Training accuracy: 99.84%

Evaluating on test set...


Testing: 100%|██████████| 44/44 [00:47<00:00,  1.07s/it]


TEST SET RESULTS
Total images: 880
Correct: 879
Accuracy: 879/880 = 99.89%
✓ Model saved to /content/face_recognition_facenet.pkl


### TESTING 2X2 COLLAGES

In [ ]:
def create_test_collages(test_folder='/content/dataset_rebuilt/test',
                        num_collages=50,
                        output_folder='/content/test_collages'):
    """Create collages for testing face recognition"""

    os.makedirs(output_folder, exist_ok=True)

    # Get all class folders
    class_folders = sorted([f for f in os.listdir(test_folder)
                           if os.path.isdir(os.path.join(test_folder, f))],
                          key=lambda x: int(x))

    print(f"Creating {num_collages} test collages from {len(class_folders)} classes...\n")

    for collage_num in range(num_collages):
        # Randomly select 4 different classes
        selected_classes = random.sample(class_folders, 4)

        images = []
        labels = []

        for class_idx in selected_classes:
            class_path = os.path.join(test_folder, class_idx)
            image_files = [f for f in os.listdir(class_path)
                          if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

            if not image_files:
                continue

            # Pick random image
            random_image = random.choice(image_files)
            img_path = os.path.join(class_path, random_image)
            img = Image.open(img_path).convert('RGB')
            images.append(img)
            labels.append(int(class_idx))

        if len(images) < 4:
            continue

        # Resize to same size
        min_width = min(img.width for img in images)
        min_height = min(img.height for img in images)
        resized_images = [img.resize((min_width, min_height), Image.LANCZOS)
                         for img in images]

        # Create 2x2 grid
        img_arrays = [np.asarray(img) for img in resized_images]
        top_row = np.hstack([img_arrays[0], img_arrays[1]])
        bottom_row = np.hstack([img_arrays[2], img_arrays[3]])
        collage = np.vstack([top_row, bottom_row])

        # Save collage
        collage_img = Image.fromarray(collage.astype('uint8'))
        output_path = os.path.join(output_folder, f'collage_{collage_num + 1:03d}.jpg')
        collage_img.save(output_path, quality=95)

        # Save ground truth labels
        labels_path = os.path.join(output_folder, f'collage_{collage_num + 1:03d}_labels.txt')
        with open(labels_path, 'w') as f:
            f.write(f"Top-left: Class {labels[0]}\n")
            f.write(f"Top-right: Class {labels[1]}\n")
            f.write(f"Bottom-left: Class {labels[2]}\n")
            f.write(f"Bottom-right: Class {labels[3]}\n")

        if (collage_num + 1) % 10 == 0:
            print(f"Created {collage_num + 1}/{num_collages} collages...")

    print(f"\n✓ Created {num_collages} collages in '{output_folder}'")

# Create collages
create_test_collages('/content/dataset_rebuilt/test',
                    num_collages=50,
                    output_folder='/content/test_collages')

Creating 50 test collages from 44 classes...

Created 10/50 collages...
Created 20/50 collages...
Created 30/50 collages...
Created 40/50 collages...
Created 50/50 collages...

✓ Created 50 collages in '/content/test_collages'


In [ ]:
def evaluate_collages_facenet(face_system, collage_folder='/content/test_collages'):
    """Evaluate FaceNet system on collages"""

    print("="*70)
    print("EVALUATING ON COLLAGES (FaceNet)")
    print("="*70)

    collage_files = sorted([f for f in os.listdir(collage_folder)
                           if f.endswith('.jpg') and f.startswith('collage_')])

    total_faces = 0
    correct_faces = 0

    for collage_file in collage_files:
        collage_path = os.path.join(collage_folder, collage_file)
        labels_path = collage_path.replace('.jpg', '_labels.txt')

        # Parse ground truth labels
        with open(labels_path, 'r') as f:
            lines = f.readlines()

        true_classes = []
        for line in lines:
            match = re.search(r'Class (\d+)', line)
            if match:
                true_classes.append(int(match.group(1)))

        # Split collage into 4 quadrants
        img = Image.open(collage_path)
        width, height = img.size
        half_w = width // 2
        half_h = height // 2

        positions = [
            (0, 0, half_w, half_h),
            (half_w, 0, width, half_h),
            (0, half_h, half_w, height),
            (half_w, half_h, width, height)
        ]

        # Predict each face
        for i, (left, top, right, bottom) in enumerate(positions):
            if i >= len(true_classes):
                break

            # Crop face
            face_crop = img.crop((left, top, right, bottom))
            temp_path = f'/content/temp_collage_face_{i}.jpg'
            face_crop.save(temp_path)

            # Predict
            pred_class, confidence = face_system.predict(temp_path)

            if pred_class is not None:
                total_faces += 1
                if pred_class == true_classes[i]:
                    correct_faces += 1

        if (collage_files.index(collage_file) + 1) % 10 == 0:
            print(f"Processed {collage_files.index(collage_file) + 1}/{len(collage_files)} collages...")

    accuracy = (correct_faces / total_faces * 100) if total_faces > 0 else 0

    print(f"\n{'='*70}")
    print("COLLAGE RESULTS (FaceNet)")
    print(f"{'='*70}")
    print(f"Total faces: {total_faces}")
    print(f"Correct: {correct_faces}")
    print(f"🎯 Accuracy: {correct_faces}/{total_faces} = {accuracy:.2f}%")
    print(f"{'='*70}")

    return accuracy

# Evaluate on collages
collage_acc = evaluate_collages_facenet(face_system, '/content/test_collages')

EVALUATING ON COLLAGES (FaceNet)
Processed 10/50 collages...
Processed 20/50 collages...
Processed 30/50 collages...
Processed 40/50 collages...
Processed 50/50 collages...

COLLAGE RESULTS (FaceNet)
Total faces: 198
Correct: 198
🎯 Accuracy: 198/198 = 100.00%


### 3X3 collage testing

In [ ]:
def create_3x3_collages(dataset_path,
                       num_collages=100,
                       split='test',
                       output_folder='/content/collages_3x3'):
    """Create 3x3 collages with 9 random faces

    Args:
        dataset_path: Path to dataset (e.g., '/content/dataset_rebuilt')
        num_collages: Number of collages to create
        split: Which split to use ('train', 'val', or 'test')
        output_folder: Where to save collages
    """

    source_dir = os.path.join(dataset_path, split)

    if not os.path.exists(source_dir):
        print(f"❌ Source folder not found: {source_dir}")
        return None

    os.makedirs(output_folder, exist_ok=True)

    # Get all class folders
    class_folders = sorted([f for f in os.listdir(source_dir)
                           if os.path.isdir(os.path.join(source_dir, f))],
                          key=lambda x: int(x))

    if len(class_folders) < 9:
        print(f"❌ Need at least 9 classes, found only {len(class_folders)}")
        return None

    print(f"Creating {num_collages} 3x3 collages from {len(class_folders)} classes...")
    print(f"Using images from: {split} set\n")

    created_count = 0

    for collage_num in range(num_collages):
        # Randomly select 9 different classes
        selected_classes = random.sample(class_folders, 9)

        images = []
        labels = []

        # Get one random image from each selected class
        for class_idx in selected_classes:
            class_path = os.path.join(source_dir, class_idx)
            image_files = [f for f in os.listdir(class_path)
                          if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

            if not image_files:
                continue

            # Pick random image
            random_image = random.choice(image_files)
            img_path = os.path.join(class_path, random_image)

            try:
                img = Image.open(img_path).convert('RGB')
                images.append(img)
                labels.append(int(class_idx))
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
                continue

        if len(images) < 9:
            print(f"⚠️  Skipping collage {collage_num + 1} - only {len(images)} valid images")
            continue

        # Resize all images to same size
        min_width = min(img.width for img in images)
        min_height = min(img.height for img in images)
        resized_images = [img.resize((min_width, min_height), Image.LANCZOS)
                         for img in images]

        # Convert to numpy arrays
        img_arrays = [np.asarray(img) for img in resized_images]

        # Create 3x3 grid
        # Row 1: images 0, 1, 2
        row1 = np.hstack([img_arrays[0], img_arrays[1], img_arrays[2]])
        # Row 2: images 3, 4, 5
        row2 = np.hstack([img_arrays[3], img_arrays[4], img_arrays[5]])
        # Row 3: images 6, 7, 8
        row3 = np.hstack([img_arrays[6], img_arrays[7], img_arrays[8]])

        # Stack all rows vertically
        collage = np.vstack([row1, row2, row3])

        # Save collage
        collage_img = Image.fromarray(collage.astype('uint8'))
        output_path = os.path.join(output_folder, f'collage_3x3_{collage_num + 1:03d}.jpg')
        collage_img.save(output_path, quality=95)

        # Save labels (positions in grid)
        labels_path = os.path.join(output_folder, f'collage_3x3_{collage_num + 1:03d}_labels.txt')
        with open(labels_path, 'w') as f:
            f.write(f"Top-left: Class {labels[0]}\n")
            f.write(f"Top-center: Class {labels[1]}\n")
            f.write(f"Top-right: Class {labels[2]}\n")
            f.write(f"Middle-left: Class {labels[3]}\n")
            f.write(f"Middle-center: Class {labels[4]}\n")
            f.write(f"Middle-right: Class {labels[5]}\n")
            f.write(f"Bottom-left: Class {labels[6]}\n")
            f.write(f"Bottom-center: Class {labels[7]}\n")
            f.write(f"Bottom-right: Class {labels[8]}\n")

        created_count += 1

        if created_count % 20 == 0:
            print(f"Created {created_count}/{num_collages} collages...")

    print(f"\n✓ Created {created_count} 3x3 collages in '{output_folder}'")
    return output_folder

# Create 100 3x3 test collages
collage_folder_3x3 = create_3x3_collages(
    dataset_path='/content/dataset_rebuilt',
    num_collages=100,
    split='test',
    output_folder='/content/collages_3x3_test'
)

Creating 100 3x3 collages from 44 classes...
Using images from: test set

Created 20/100 collages...
Created 40/100 collages...
Created 60/100 collages...
Created 80/100 collages...
Created 100/100 collages...

✓ Created 100 3x3 collages in '/content/collages_3x3_test'


In [ ]:
def evaluate_3x3_collages_facenet(face_system, collage_folder='/content/collages_3x3_test'):
    """Evaluate FaceNet on 3x3 collages"""

    print("="*70)
    print("EVALUATING ON 3x3 COLLAGES (9 Faces)")
    print("="*70)

    collage_files = sorted([f for f in os.listdir(collage_folder)
                           if f.endswith('.jpg') and 'collage_3x3' in f])

    if not collage_files:
        print(f"❌ No collages found in {collage_folder}")
        return 0

    print(f"Found {len(collage_files)} collages\n")

    total_faces = 0
    correct_faces = 0

    for collage_file in collage_files:
        collage_path = os.path.join(collage_folder, collage_file)
        labels_path = collage_path.replace('.jpg', '_labels.txt')

        if not os.path.exists(labels_path):
            print(f"⚠️  No labels for {collage_file}")
            continue

        # Parse ground truth labels
        with open(labels_path, 'r') as f:
            lines = f.readlines()

        true_classes = []
        for line in lines:
            match = re.search(r'Class (\d+)', line)
            if match:
                true_classes.append(int(match.group(1)))

        if len(true_classes) != 9:
            print(f"⚠️  Expected 9 labels, got {len(true_classes)} for {collage_file}")
            continue

        # Load and split collage into 3x3 grid
        img = Image.open(collage_path)
        width, height = img.size

        cell_width = width // 3
        cell_height = height // 3

        # Define 9 positions (3x3 grid)
        positions = []
        for row in range(3):
            for col in range(3):
                left = col * cell_width
                top = row * cell_height
                right = left + cell_width
                bottom = top + cell_height
                positions.append((left, top, right, bottom))

        # Predict each face
        for i, (left, top, right, bottom) in enumerate(positions):
            # Crop face
            face_crop = img.crop((left, top, right, bottom))
            temp_path = f'/content/temp_3x3_face_{i}.jpg'
            face_crop.save(temp_path)

            # Predict
            pred_class, confidence = face_system.predict(temp_path)

            if pred_class is not None:
                total_faces += 1
                if pred_class == true_classes[i]:
                    correct_faces += 1

        if (collage_files.index(collage_file) + 1) % 20 == 0:
            print(f"Processed {collage_files.index(collage_file) + 1}/{len(collage_files)} collages...")

    accuracy = (correct_faces / total_faces * 100) if total_faces > 0 else 0

    print(f"\n{'='*70}")
    print("3x3 COLLAGE RESULTS")
    print(f"{'='*70}")
    print(f"Total collages: {len(collage_files)}")
    print(f"Total faces: {total_faces}")
    print(f"Correct predictions: {correct_faces}")
    print(f"Incorrect predictions: {total_faces - correct_faces}")
    print(f"\n🎯 Accuracy: {correct_faces}/{total_faces} = {accuracy:.2f}%")
    print(f"{'='*70}")

    return accuracy

# Evaluate on 3x3 collages
if os.path.exists('/content/collages_3x3_test'):
    collage_3x3_acc = evaluate_3x3_collages_facenet(face_system, '/content/collages_3x3_test')

EVALUATING ON 3x3 COLLAGES (9 Faces)
Found 100 collages

Processed 20/100 collages...
Processed 40/100 collages...
Processed 60/100 collages...
Processed 80/100 collages...
Processed 100/100 collages...

3x3 COLLAGE RESULTS
Total collages: 100
Total faces: 891
Correct predictions: 890
Incorrect predictions: 1

🎯 Accuracy: 890/891 = 99.89%


### SAVING the model in DRIVE

In [ ]:


def save_complete_progress(backup_name=None):
    """Save all progress to Google Drive"""

    # Mount Google Drive
    drive.mount('/content/drive')

    # Create backup folder with timestamp
    if backup_name is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        backup_name = f'face_recognition_backup_{timestamp}'

    backup_root = f'/content/drive/MyDrive/{backup_name}'
    os.makedirs(backup_root, exist_ok=True)

    print("="*70)
    print(f"SAVING COMPLETE PROGRESS TO GOOGLE DRIVE")
    print("="*70)
    print(f"Backup location: {backup_root}\n")

    saved_items = []

    # 1. Save FaceNet model
    print("1. Saving FaceNet model...")
    if os.path.exists('/content/face_recognition_facenet.pkl'):
        shutil.copy2('/content/face_recognition_facenet.pkl',
                    f'{backup_root}/face_recognition_facenet.pkl')
        print("   ✓ FaceNet model saved")
        saved_items.append('FaceNet model')
    else:
        print("   ⚠️  FaceNet model not found")

    # 2. Save dataset
    print("\n2. Saving dataset...")
    dataset_paths = [
        '/content/dataset_rebuilt',
        '/content/dataset_yolo_final',
        '/content/face_recognition_dataset'
    ]

    dataset_saved = False
    for dataset_path in dataset_paths:
        if os.path.exists(dataset_path):
            dataset_name = os.path.basename(dataset_path)
            dest = f'{backup_root}/{dataset_name}'

            print(f"   Copying {dataset_name}...")
            if os.path.exists(dest):
                shutil.rmtree(dest)
            shutil.copytree(dataset_path, dest)

            # Get size
            size = sum(os.path.getsize(os.path.join(dirpath, filename))
                      for dirpath, dirnames, filenames in os.walk(dest)
                      for filename in filenames)
            size_mb = size / (1024 * 1024)

            print(f"   ✓ Dataset saved: {dataset_name} ({size_mb:.2f} MB)")
            saved_items.append(f'Dataset ({dataset_name})')
            dataset_saved = True
            break

    if not dataset_saved:
        print("   ⚠️  No dataset found")

    # 3. Save all collages
    print("\n3. Saving collages...")
    collage_folders = [
        '/content/collages_3x3',
        '/content/collages_2x2',
        '/content/collages_4x4',
        '/content/test_collages',
        '/content/collages'
    ]

    collage_count = 0
    for collage_folder in collage_folders:
        if os.path.exists(collage_folder):
            folder_name = os.path.basename(collage_folder)
            dest = f'{backup_root}/collages/{folder_name}'

            os.makedirs(os.path.dirname(dest), exist_ok=True)
            if os.path.exists(dest):
                shutil.rmtree(dest)
            shutil.copytree(collage_folder, dest)

            num_collages = len([f for f in os.listdir(dest) if f.endswith('.jpg')])
            print(f"   ✓ Saved {folder_name}: {num_collages} collages")
            collage_count += num_collages

    if collage_count > 0:
        saved_items.append(f'Collages ({collage_count} total)')
    else:
        print("   ⚠️  No collages found")

    # 4. Save class mapping
    print("\n4. Saving class mapping...")
    mapping_files = [
        '/content/dataset_rebuilt/class_mapping.json',
        '/content/dataset_yolo_final/class_mapping.json',
        '/content/identity_mapping.json'
    ]

    for mapping_file in mapping_files:
        if os.path.exists(mapping_file):
            filename = os.path.basename(mapping_file)
            shutil.copy2(mapping_file, f'{backup_root}/{filename}')
            print(f"   ✓ Saved {filename}")
            saved_items.append('Class mapping')
            break

    # 5. Save training results (if from YOLO)
    print("\n5. Saving training results...")
    if os.path.exists('runs'):
        dest = f'{backup_root}/training_runs'
        if os.path.exists(dest):
            shutil.rmtree(dest)
        shutil.copytree('runs', dest)
        print("   ✓ Training runs saved")
        saved_items.append('Training results')

    # 6. Save metadata
    print("\n6. Saving metadata...")
    metadata = {
        'backup_date': datetime.now().isoformat(),
        'saved_items': saved_items,
        'python_version': '3.12',
        'libraries': {
            'facenet_pytorch': 'installed',
            'ultralytics': 'installed',
            'torch': 'installed'
        }
    }

    with open(f'{backup_root}/backup_metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    print("   ✓ Metadata saved")

    # 7. Save restore instructions
    restore_instructions = f"""
# ========================================
# RESTORE INSTRUCTIONS
# ========================================

# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Run the restore script (see restore_progress.py)
# Or manually copy files:

# Restore dataset:
import shutil
shutil.copytree('{backup_root}/dataset_rebuilt', '/content/dataset_rebuilt')

# Restore model:
shutil.copy2('{backup_root}/face_recognition_facenet.pkl',
             '/content/face_recognition_facenet.pkl')

# Restore collages:
shutil.copytree('{backup_root}/collages', '/content')

# 3. Load the model:
from facenet_pytorch import MTCNN, InceptionResnetV1
import pickle

# Load saved model
with open('/content/face_recognition_facenet.pkl', 'rb') as f:
    model_data = pickle.load(f)

# Reinitialize FaceRecognitionSystem and load weights
face_system = FaceRecognitionSystem()
face_system.classifier = model_data['classifier']
face_system.label_encoder = model_data['label_encoder']
face_system.normalizer = model_data['normalizer']

print("✓ Ready to continue!")
"""

    with open(f'{backup_root}/RESTORE_INSTRUCTIONS.txt', 'w') as f:
        f.write(restore_instructions)

    print("\n" + "="*70)
    print("BACKUP COMPLETE!")
    print("="*70)
    print(f"Location: {backup_root}")
    print(f"\nSaved items:")
    for item in saved_items:
        print(f"  • {item}")
    print(f"\nTo restore, see: {backup_root}/RESTORE_INSTRUCTIONS.txt")
    print("="*70)

    return backup_root

# Save everything
backup_location = save_complete_progress()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
SAVING COMPLETE PROGRESS TO GOOGLE DRIVE
Backup location: /content/drive/MyDrive/face_recognition_backup_20251006_011813

1. Saving FaceNet model...
   ✓ FaceNet model saved

2. Saving dataset...
   Copying dataset_rebuilt...
   ✓ Dataset saved: dataset_rebuilt (56.35 MB)

3. Saving collages...
   ✓ Saved test_collages: 50 collages

4. Saving class mapping...
   ✓ Saved class_mapping.json

5. Saving training results...

6. Saving metadata...
   ✓ Metadata saved

BACKUP COMPLETE!
Location: /content/drive/MyDrive/face_recognition_backup_20251006_011813

Saved items:
  • FaceNet model
  • Dataset (dataset_rebuilt)
  • Collages (50 total)
  • Class mapping

To restore, see: /content/drive/MyDrive/face_recognition_backup_20251006_011813/RESTORE_INSTRUCTIONS.txt
